In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import cv2
import torch
import torch.nn as nn
from PIL import Image

In [0]:
####
#the data 

x1 = np.array([[0,0,0],[0,0,0],[0,0,0]])
x2 = np.array([[1,1,1],[0,0,0],[0,0,0]])
x3 = np.array([[0,0,0],[1,1,1],[1,1,1]])
x4 = np.array([ [1,1,1],[1,1,1],[1,1,1]])
X = [x1,x2,x3,x4]
Y = np.array([
    [0.53],
    [0.77],
    [0.88],
    [1.1]
])

X = torch.from_numpy(np.asarray(X)).float()
Y = torch.from_numpy(np.asarray(Y)).float()

In [0]:
!python --version

Python 3.6.7


In [0]:
#! Task 1, complete the torch cnn class below. 
# make 2 layer network with 2x2 conv with activation tanh followed by 
# fullyconnected with sigmoid activation

class CNNFC(nn.Module):
    
    def __init__(self, num_classes=10):
        super().__init__()
        default_num_filters = 8
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, default_num_filters, kernel_size=2),
            nn.Tanh(),
        )
        
        self.fc = nn.Sequential(
            nn.Linear(32, num_classes),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        #dont forget to reshape your output of CNN to be one vector
        #ex : out.reshape(out.size(0), -1)
        out = self.conv1(x)
        out = out.reshape(out.size(0), -1)
        print(out.size())
        out = self.fc(out)
        return out


CNN = CNNFC()
criterion = nn.MSELoss()

### Explanation
Here we can see the 2 by 2 filters of our CNN, and since we have 8 filters, we see 8 2-by-2 matrix and each one has its own weight.

We are using `cnn[0]` because we want the parameters of the first layer.

In [0]:
list(CNN.conv1[0].parameters())

[Parameter containing:
 tensor([[[[-0.2586, -0.3064],
           [ 0.4201, -0.0662]]],
 
 
         [[[-0.0859, -0.1301],
           [-0.2631, -0.0312]]],
 
 
         [[[ 0.3590,  0.4374],
           [-0.4311,  0.4236]]],
 
 
         [[[-0.3382,  0.0993],
           [-0.3229, -0.3257]]],
 
 
         [[[-0.0050, -0.0903],
           [-0.1599,  0.3080]]],
 
 
         [[[ 0.2291,  0.1617],
           [-0.4660,  0.0994]]],
 
 
         [[[ 0.4560,  0.3683],
           [ 0.2122,  0.0939]]],
 
 
         [[[-0.0896, -0.2397],
           [ 0.4120,  0.1359]]]], requires_grad=True), Parameter containing:
 tensor([-0.1253,  0.1065,  0.4337, -0.4835, -0.3036, -0.3609, -0.0567,  0.2023],
        requires_grad=True)]

In [0]:
#! Task 2, copy the old weight filter of CNN using np.copy()

cw1 = np.copy(list(CNN.conv1.parameters()))
cw2 = np.copy(list(CNN.fc.parameters()))

#! Task 3, define the learning rate and optimizer 
learning_rate = 0.01
optimizer = torch.optim.Adam(params=CNN.parameters(), lr=learning_rate)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_epoch = 5


#! task 4 compute cost prior training 
cost_before_train = 0

for i in range(len(X)):
    CNN.eval()
    input = X[i].unsqueeze(0).unsqueeze(0)
    
    out = CNN(input)
    
    cost = np.square(- Y[i]).sum() * 0.5
    cost_before_train = cost_before_train + cost

torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])


In [0]:
#! Task 5 do the training 
CNN.train()

for iter in range(num_epoch):
    for i in range(len(X)):
        out = CNN(X[i].unsqueeze(0).unsqueeze(0))
        loss = criterion(out, Y[i])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [0]:
#! Task 6, compute Cost after training
cost_after_train = 0
for i in range(len(X)):
    CNN.eval()
    with torch.no_grad(): 
        input = X[i].unsqueeze(0).unsqueeze(0)
        out = CNN(input)
        cost = np.square(out- Y[i]).sum() * 0.5
        cost_after_train = cost_after_train + cost


# ----- Print Results ---
print(cw1,cw2)

w1 = np.copy(list(CNN.conv1.parameters()))
w2 = np.copy(list(CNN.fc.parameters()))


print("\nW1 :",w1, "\n\nw2 :", w2)
print("----------------")
print("Cost before Training: ",cost_before_train)
print("Cost after Training: ",cost_after_train)
print("----------------")

torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
[Parameter containing:
tensor([[[[-0.3807, -0.4285],
          [ 0.3075, -0.1788]]],


        [[[-0.2112, -0.2554],
          [-0.3738, -0.1419]]],


        [[[ 0.3945,  0.4728],
          [-0.3208,  0.5340]]],


        [[[-0.2939,  0.1436],
          [-0.2896, -0.2924]]],


        [[[-0.0603, -0.1455],
          [-0.2466,  0.2213]]],


        [[[ 0.1091,  0.0417],
          [-0.5512,  0.0142]]],


        [[[ 0.4334,  0.3457],
          [ 0.2599,  0.1417]]],


        [[[ 0.0139, -0.1361],
          [ 0.5051,  0.2290]]]], requires_grad=True)
 Parameter containing:
tensor([-0.2558, -0.0380,  0.4273, -0.3781, -0.3103, -0.4322, -0.1078,  0.2578],
       requires_grad=True)] [Parameter containing:
tensor([[-2.3421e-01, -1.1550e-01, -7.7451e-02, -1.2844e-01, -4.6426e-02,
         -2.3073e-01, -2.4206e-01, -1.0519e-01,  2.0358e-01, -4.6927e-02,
         -1.3397e-02,  9.4828e-02, -2.1346e-01, -1.2389e-01, -3